# **Подготовка данных**

Для начала нам необходимо загрузить файлы в виде pandas DataFrame

In [1]:
# Загрузка локальных файлов

from google.colab import files
import io

uploaded = files.upload()

Saving musae_git_edges.csv to musae_git_edges (10).csv
Saving musae_git_target.csv to musae_git_target (10).csv


In [2]:
# Импорт модуля pandas
import pandas as pd

# Чтение csv-файлов
target = pd.read_csv(io.BytesIO(uploaded['musae_git_target.csv']))
edges = pd.read_csv(io.BytesIO(uploaded['musae_git_edges.csv']))

## **Работа с узлами** *(musae_git_target.csv)*

In [3]:
target.head()

,id,name,ml_target
0,0,Eiryyy,0
1,1,shawflying,0
2,2,JpMCarrilho,1
3,3,SuhwanCha,0
4,4,sunilangadi2,1


Для большего разнообразия добавим пользователям дополнительные характеристики: пол и возраст. Согласно данным из открытых источников, женщин и мужчин в IT 34% и 66% соответственно, тогда как возраст разработчиков:
*   18-24 -> 19,8%
*   25-34 -> 63,9%
*   35-44 -> 14,1%
*   45+ -> 2,2%





In [4]:
import random

target["sex"] = random.choices(["M", "F"], weights=[0.66, 0.34], k=target.shape[0])
target

,id,name,ml_target,sex
0,0,Eiryyy,0,M
1,1,shawflying,0,M
2,2,JpMCarrilho,1,M
3,3,SuhwanCha,0,F
4,4,sunilangadi2,1,M
...,...,...,...,...
9996,9996,denpalrius,0,M
9997,9997,lijunjieone,0,F
9998,9998,leocvml,1,M
9999,9999,ArsalaBangash,1,M


In [5]:
target["age"] = random.choices([random.randint(18, 24), random.randint(25, 34), random.randint(35, 44), random.randint(45, 80)], weights=[0.198, 0.639, 0.141, 0.22], k=target.shape[0])
target

,id,name,ml_target,sex,age
0,0,Eiryyy,0,M,42
1,1,shawflying,0,M,18
2,2,JpMCarrilho,1,M,32
3,3,SuhwanCha,0,F,42
4,4,sunilangadi2,1,M,32
...,...,...,...,...,...
9996,9996,denpalrius,0,M,42
9997,9997,lijunjieone,0,F,71
9998,9998,leocvml,1,M,42
9999,9999,ArsalaBangash,1,M,71


Проверяем, что все данные заполнены.

In [6]:
target.isna().sum()

id           0
name         0
ml_target    0
sex          0
age          0
dtype: int64

## **Работа со связями** *(musae_git_edges.csv)*

In [7]:
edges.head()

,id_1,id_2
0,1,237
1,1,2383
2,6067,1972
3,3,495
4,3,3358


Исходя из описания данных, связи представляют собой подписки пользователей друг на друга, то есть - ребра неориентированного графа. Так как в дальнейшем эти данные будут использоваться для построения ориентированного графа, необходимо добавить несколько строк, обратных имеющимся, чтобы отобразить взаимную подписку пользователей друг на друга. Возьмем 25% строк случайным образом и поменяем названия колонок местами.

In [8]:
reverse_edges = edges.sample(frac=0.25, random_state=1).rename(columns={"id_1": "id_2", "id_2": "id_1"})
reverse_edges.tail()

,id_2,id_1
12813,3680,9544
19083,6005,3712
9564,2635,6854
7291,2078,2105
21688,6923,3615


Добавим полученные строки в конец DataFrame

In [9]:
edges = pd.concat([edges, reverse_edges], ignore_index=True)
edges.head()

,id_1,id_2
0,1,237
1,1,2383
2,6067,1972
3,3,495
4,3,3358


In [10]:
edges.tail()

,id_1,id_2
36176,9544,3680
36177,3712,6005
36178,6854,2635
36179,2105,2078
36180,3615,6923


Начало таблицы осталось прежним, а окончание совпадает с последними строками reverse_edges.

Далее необходимо создать матрицу смежности в виде DataFrame, в котором строки содержат id пользователя-подписчика, id пользователя, на которого подписались, и значение типа bool в зависимости от того, существует ли такая подписка или нет. Для этого добавим в DataFrame существующих связей колонку edge, заполнив ее единицами.

In [11]:
edges["edge"] = 1
edges

,id_1,id_2,edge
0,1,237,1
1,1,2383,1
2,6067,1972,1
3,3,495,1
4,3,3358,1
...,...,...,...
36176,9544,3680,1
36177,3712,6005,1
36178,6854,2635,1
36179,2105,2078,1


Создадим матрицу смежности в виде DataFrame из всех возможных пар пользователей.

In [12]:
adjacency = pd.DataFrame(target["id"])
adjacency = adjacency.merge(adjacency, how="cross")
adjacency

,id_x,id_y
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
...,...,...
100019996,10000,9996
100019997,10000,9997
100019998,10000,9998
100019999,10000,9999


И объединим новую таблицу с таблицей существующих связей, взяв от результата merge список всех пар id и связи.

In [13]:
adjacency = adjacency.merge(edges, how="left", left_on=["id_x", "id_y"], right_on=["id_1", "id_2"], copy=False)
adjacency = adjacency[['id_x', 'id_y', 'edge']]
adjacency

,id_x,id_y,edge
0,0,0,NaN
1,0,1,NaN
2,0,2,NaN
3,0,3,NaN
4,0,4,NaN
...,...,...,...
100020023,10000,9996,NaN
100020024,10000,9997,NaN
100020025,10000,9998,NaN
100020026,10000,9999,NaN


У пар, данных о которых не было в исходном файле связей, образовалось значение NaN. Его необходимо заменить на 0.

In [14]:
adjacency["edge"] = adjacency["edge"].fillna(0)
adjacency

,id_x,id_y,edge
0,0,0,0.0
1,0,1,0.0
2,0,2,0.0
3,0,3,0.0
4,0,4,0.0
...,...,...,...
100020023,10000,9996,0.0
100020024,10000,9997,0.0
100020025,10000,9998,0.0
100020026,10000,9999,0.0


Проверяем, что количество единиц в полученном DataFrame совпадает с числом существующих связей в edges.

In [15]:
adjacency["edge"].value_counts()[1] == edges.shape[0]

True

Проверяем, что все данные заполнены.

In [16]:
adjacency.isna().sum()

id_x    0
id_y    0
edge    0
dtype: int64